# AI cập nhật tin tức mới nhất cho báo cáo tuần

In [10]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import get_and_crawl_data
import gemini_model
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(get_and_crawl_data)
importlib.reload(gemini_model)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from get_and_crawl_data import *
from gemini_model import *
from plotly_and_upload import *

In [11]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))

# Sắp xếp danh sách model theo thứ tự ổn định
fast_model_list = select_fast_models(get_gemini_models())
standard_model_list = select_standard_models(get_gemini_models())

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

In [12]:
vietstock_rss_url_dict = {
    'trong_nuoc': {
        'vi_mo': 'https://vietstock.vn/761/kinh-te/vi-mo.rss',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss',
        'chung_khoan_the_gioi': 'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss',
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss',
    }
}

cafef_rss_url_dict = {
    'trong_nuoc': {
        'kinh_te_vi_mo': 'https://cafef.vn/vi-mo-dau-tu.chn',
        'chung_khoan': 'https://cafef.vn/thi-truong-chung-khoan.chn',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://cafef.vn/tai-chinh-quoc-te.chn',
    },
    'doanh_nghiep': {
        'doanh_nghiep': 'https://cafef.vn/doanh-nghiep.chn',
    }
}

vietnambiz_rss_url_dict = {
    'trong_nuoc': {
        'chung_khoan': 'https://vietnambiz.vn/chung-khoan/thi-truong.htm',
        'nha_dat': 'https://vietnambiz.vn/nha-dat/thi-truong.htm',
    },
    'quoc_te': {
        'tai_chinh_chung_khoan': 'https://vietnambiz.vn/quoc-te/tai-chinh-chung-khoan.htm',
    },
    'doanh_nghiep': {
        'ket_qua_kinh_doanh': 'https://vietnambiz.vn/doanh-nghiep/ket-qua-kinh-doanh.htm',
    }
}

In [13]:
vietstock_count_dict = {
    'trong_nuoc': {
        'vi_mo': 4,
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 4,
        'chung_khoan_the_gioi': 4,
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 8,
    }
}

cafef_count_dict = {
    'trong_nuoc': {
        'kinh_te_vi_mo': 4,
        'chung_khoan': 4,
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 6,
    },
    'doanh_nghiep': {
        'doanh_nghiep': 6,
    }
}

vietnambiz_count_dict = {
    'trong_nuoc': {
        'chung_khoan': 4,
        'nha_dat': 4,
    },
    'quoc_te': {
        'tai_chinh_chung_khoan': 6
    },
    'doanh_nghiep': {
        'ket_qua_kinh_doanh': 6
    }
}

In [14]:
full_news_df_dict = {}

# Khởi tạo dictionary với list rỗng cho mỗi news_type
for news_type in ['trong_nuoc', 'quoc_te', 'doanh_nghiep']:
    full_news_df_dict[news_type] = []

# Lấy tin từ VietStock
for news_type, url_dict in vietstock_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = vietstock_count_dict[news_type][category_name]
        feed = feedparser.parse(url)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed.entries[:max_entries]:
            # Get both content and image URL
            content, image_url = get_article_vietstock(entry['id'])
            
            # Lấy thời gian đăng bài từ RSS feed
            published_time = ""
            if hasattr(entry, 'published') and entry.published:
                published_time = entry.published
            
            full_news_df_dict[news_type].append({
                'source': 'VietStock',
                'title': entry['title'], 
                'content': summary_weekly_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)

# Lấy tin từ CafeF
for news_type, url_dict in cafef_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = cafef_count_dict[news_type][category_name]
        # Lấy danh sách bài viết từ trang danh mục (tạo feed giả)
        feed_entries = get_cafef_articles_list(url, max_entries)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed_entries[:max_entries]:
            # Get both content and image URL (giống VietStock)
            content, image_url = get_article_cafef(entry['id'])
            
            # Lấy thời gian đăng bài từ bài viết
            published_time = get_cafef_published_time(entry['id'])
            
            full_news_df_dict[news_type].append({
                'source': 'CafeF',
                'title': entry['title'], 
                'content': summary_weekly_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)

# Lấy tin từ Vietnambiz 
for news_type, url_dict in vietnambiz_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = vietnambiz_count_dict[news_type][category_name]
        # Lấy danh sách bài viết từ trang danh mục (tạo feed giả)
        feed_entries = get_vietnambiz_articles_list(url, max_entries)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed_entries[:max_entries]:
            # Get both content and image URL (giống VietStock)
            content, image_url = get_article_vietnambiz(entry['id'])
            
            # Lấy thời gian đăng bài từ bài viết
            published_time = get_vietnambiz_published_time(entry['id'])
            
            full_news_df_dict[news_type].append({
                'source': 'Vietnambiz',
                'title': entry['title'], 
                'content': summary_weekly_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)

LỖI: Model 'gemini-2.0-flash' gặp lỗi nghiêm trọng: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 17
}
].
LỖI: Model 'gemini-2.0-flash' gặp lỗi nghiêm trọng: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 30
}
].
LỖI: Model 'gemini-2.0-flash' gặp lỗi nghiêm trọng: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.googl

In [15]:
# Chuyển đổi dictionary thành pandas DataFrame
all_news_list = []
for news_type, articles in full_news_df_dict.items():
    for article in articles:
        # Thêm cột news_type vào mỗi article
        article_with_type = article.copy()
        article_with_type['news_type'] = news_type
        all_news_list.append(article_with_type)

# Tạo DataFrame
weekly_news_list_df = pd.DataFrame(all_news_list)
weekly_news_list_df['published_time'] = weekly_news_list_df['published_time'].apply(convert_published_time)
weekly_news_list_df = weekly_news_list_df[['news_type'] + [col for col in weekly_news_list_df.columns if col != 'news_type']]

# Thêm các cột cần thiết
weekly_news_list_df['word_count'] = weekly_news_list_df['content'].str.split().str.len()
weekly_news_list_df['impact'] = analyze_news_impact(standard_model_dict, weekly_news_list_df)
weekly_news_list_df['sectors'] = analyze_news_sectors(standard_model_dict, weekly_news_list_df)

CẢNH BÁO: Model 'gemini-2.5-flash' lỗi lần 0/2. Lý do: Unknown.


In [16]:
def create_comment_prompt(df, news_type):
    prompt = f"""
    Bạn là một chuyên viên phân tích thị trường. Dưới đây là bảng dữ liệu các tin tức ngắn, mỗi dòng gồm các trường: [title], [content], [impact], [sectors].

    Nhiệm vụ của bạn:
    - Viết một đoạn văn duy nhất, súc tích, tổng hợp toàn bộ thông tin trong bảng.
    - Đoạn văn phải tuân thủ NGHIÊM NGẶT các quy tắc sau:

    1. Độ dài:
        - Đúng 5 câu.
        - Mỗi câu dài từ 13 đến 15 từ.

    2. Cấu trúc nội dung:
        - Câu 1: Nêu nhận định hoặc xu hướng chính nổi bật nhất từ các tin tức.
        - Câu 2: Trình bày nguyên nhân, động lực hoặc yếu tố tích cực quan trọng nhất hỗ trợ xu hướng đó.
        - Câu 3: Đề cập một khó khăn, rủi ro hoặc thông tin trái ngược mang tính kìm hãm.
        - Câu 4: Mô tả kết quả, hệ quả hoặc ảnh hưởng thực tế đến một lĩnh vực/đối tượng cụ thể.
        - Câu 5: Tổng hợp các ý trên để đưa ra kết luận chung hoặc dự báo xu hướng sắp tới.

    3. Yêu cầu khác:
        - Chỉ sử dụng thông tin trong bảng, không thêm ý ngoài.
        - Tuyệt đối không đề cập đến 1 tin tức cụ thể nào.
        - Chủ đề tập trung vào kinh tế vĩ mô, không quá tập trung vào thị trường chứng khoán.
        - Văn phong khách quan, cân bằng, không cảm tính.
        - Chỉ hiển thị đoạn văn hoàn chỉnh, không lặp lại hướng dẫn.

    Bảng dữ liệu:
    {df[df['news_type']==news_type][['title', 'content', 'impact', 'sectors']].to_csv(index=False, sep='|', lineterminator='\\n')}
    """

    return prompt

trong_nuoc_comment = generate_content_with_model_dict(standard_model_dict, create_comment_prompt(weekly_news_list_df, 'trong_nuoc'))
quoc_te_comment = generate_content_with_model_dict(standard_model_dict, create_comment_prompt(weekly_news_list_df, 'quoc_te'))
doanh_nghiep_comment = generate_content_with_model_dict(standard_model_dict, create_comment_prompt(weekly_news_list_df, 'doanh_nghiep'))
weekly_news_comments_df = pd.DataFrame({
    'news_type': ['trong_nuoc', 'quoc_te', 'doanh_nghiep'],
    'comment': [trong_nuoc_comment, quoc_te_comment, doanh_nghiep_comment],
})

In [17]:
%%capture
save_to_mssql(cts_engine, weekly_news_list_df, 'weekly_news_list')
save_to_mssql(cts_engine, weekly_news_comments_df, 'weekly_news_comments')